<a href="https://colab.research.google.com/github/VrijKun/CS6910_Assignment_2/blob/main/Assignment_2_DL_ED23D015.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In Part A and Part B of this assignment you will build and experiment with CNN based image classifiers using a subset of the iNaturalist dataset. In Part C you will take a pre-trained object detection model and use it for a novel application.

In [1]:
pip install --upgrade pytorch-lightning

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as functionals
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import pytorch_lightning as pyligtining
print(pyligtining.__version__)

2.2.1


#Question 1 (5 Marks)
Build a small CNN model consisting of 5 convolution layers. Each convolution layer would be followed by a ReLU activation and a max pooling layer. Here is sample code for building one such conv-relu-maxpool block in keras.

model = Sequential()
model.add(Conv2D(16, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
After 5 such conv-relu-maxpool blocks of layers you should have one dense layer followed by the output layer containing 10 neurons (1 for each of the 10 classes). The input layer should be compatible with the images in the iNaturalist dataset.

**The code should be flexible such that the number of filters, size of filters and activation function in each layer can be changed. You should also be able to change the number of neurons in the dense layer.**

(a) What is the total number of computations done by your network? (assume mmm filters in each layer of size k×kk\times kk×k and nnn neurons in the dense layer)

(b) What is the total number of parameters in your network? (assume mmm filters in each layer of size k×kk\times kk×k and nnn neurons in the dense layer)


In [3]:
!pip install wget
import wget
wget.download('https://storage.googleapis.com/wandb_datasets/nature_12K.zip')
!unzip /content/nature_12K.zip

Archive:  /content/nature_12K.zip
replace inaturalist_12K/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
from torchvision import datasets, transforms
import pytorch_lightning as pl


class ECNN(pl.LightningModule):
    def __init__(self, input_shape, num_classes, num_filters, filter_size, dense_neurons, batch_normalization, batch_size, dropout, activation):
        super().__init__()
        self.input_shape = input_shape
        #print(self.input_shape)
        self.num_classes = num_classes
        self.num_filters = num_filters
        self.filter_size = filter_size
        self.dense_neurons = dense_neurons
        self.activation = activation
        self.batch_normalization=batch_normalization

        # Define convolutional layers
        self.conv1 = nn.Conv2d(3, num_filters[0], filter_size[0])
        self.conv2 = nn.Conv2d(num_filters[0], num_filters[1], filter_size[1])
        self.conv3 = nn.Conv2d(num_filters[1], num_filters[2], filter_size[2])
        self.conv4 = nn.Conv2d(num_filters[2], num_filters[3], filter_size[3])
        self.conv5 = nn.Conv2d(num_filters[3], num_filters[4], filter_size[4])

        # Defining batch normalization
        self.bn1 = nn.BatchNorm2d(num_filters[0])
        self.bn2 = nn.BatchNorm2d(num_filters[1])
        self.bn3 = nn.BatchNorm2d(num_filters[2])
        self.bn4 = nn.BatchNorm2d(num_filters[3])
        self.bn5 = nn.BatchNorm2d(num_filters[4])


        # Define pooling layer
        self.pool = nn.MaxPool2d(2, 2)

        # Define dropout
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)
        self.dropout4 = nn.Dropout(dropout)
        self.dropout5 = nn.Dropout(dropout)

        # Calculate the size of the feature maps after max pooling
        self.flatten_size = self.calculate_flatten_size()

        # Define dense layers
        self.fc1 = nn.Linear(self.flatten_size, dense_neurons)
        self.fc2 = nn.Linear(dense_neurons, num_classes)

    '''
    ########################################
    def forward(self, x):
        for conv, bn in zip(self.conv_layers, self.bn_layers):
            x = F.relu(bn(conv(x)))
            x = self.pool(x)
        x = x.view(-1, self.flatten_size)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

    def calculate_flatten_size(self):
        # Dummy input to calculate the size of the feature maps after max pooling
        x = torch.randn(1, *self.input_shape)
        for conv, bn in zip(self.conv_layers, self.bn_layers):
            x = F.relu(bn(conv(x)))
            x = self.pool(x)
        return x.view(x.size(0), -1).size(1)

    #####################################
    '''

    def forward(self, x):
        if self.batch_normalization:
          x = F.relu(self.bn1(self.conv1(x)))
        else:
          x = F.relu(self.conv1(x))
        x=self.dropout1(x)
        x = self.pool(x)
        #print('0: ',x.shape)

        if self.batch_normalization:
          x = F.relu(self.bn2(self.conv2(x)))
        else:
          x = F.relu(self.conv2(x))
        x=self.dropout2(x)
        x = self.pool(x)
        #print('1: ',x.shape)

        if self.batch_normalization:
          x = F.relu(self.bn3(self.conv3(x)))
        else:
          x = F.relu(self.conv3(x))
        x=self.dropout3(x)
        x = self.pool(x)
        #print('2: ',x.shape)

        if self.batch_normalization:
          x = F.relu(self.bn4(self.conv4(x)))
        else:
          x = F.relu(self.conv4(x))
        x=self.dropout4(x)
        x = self.pool(x)
        #print('3: ',x.shape)

        if self.batch_normalization:
          x = F.relu(self.bn5(self.conv5(x)))
        else:
          x = F.relu(self.conv5(x))
        x=self.dropout5(x)
        x = self.pool(x)


        x = x.view(x.size(0), -1)
        #print('5: ',x.shape)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        #print('5.5: ',x.shape)
        x = F.log_softmax(x, dim=1) # trying if it works
        #return F.log_softmax(x, dim=1)
        return x

    def calculate_flatten_size(self):

        x = torch.randn(1, *self.input_shape)
        x = self.pool(self.conv1(x))
        x = self.pool(self.conv2(x))
        x = self.pool(self.conv3(x))
        x = self.pool(self.conv4(x))
        x = self.pool(self.conv5(x))
        return x.view(x.size(0), -1).size(1)

        '''
        print('3: ',x.shape)
        x = self.pool(self.conv4(x))
        print('4: ',x.shape)
        x = self.pool(self.conv5(x))
        print('5: ',x.shape)
        return x.view(x.size(0), -1).size(1)
        '''

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

    def training_step(self, train_loader):
        x, y = train_loader
        y_hat = self(x)
        loss = F.nll_loss(y_hat, y)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, val_loader):
        x, y = val_loader
        y_hat = self(x)
        loss = F.nll_loss(y_hat, y)
        self.log('val_loss', loss)

    def test_step(self, test_loader):
        x, y = test_loader
        y_hat = self(x)
        loss = F.nll_loss(y_hat, y)
        self.log('test_loss', loss)

    def accuracy(y_hat, y):
        preds = torch.argmax(y_hat, dim=1)
        correct = (preds == y).sum().item()
        total = y.size(0)
        acc = correct / total
        return acc

    def validation_accuracy(model, val_loader):
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for batch in val_loader:
                x, y = batch
                y_hat = model(x)
                correct += (torch.argmax(y_hat, dim=1) == y).sum().item()
                total += y.size(0)
        acc = correct / total
        return acc

    def test_accuracy(model, test_loader):
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for batch in test_loader:
                x, y = batch
                y_hat = model(x)
                correct += (torch.argmax(y_hat, dim=1) == y).sum().item()
                total += y.size(0)
        acc = correct / total
        return acc

    '''
    def on_validation_epoch_end(self, x): # outputs
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        self.log('val_loss', avg_loss)
        val_acc = self.validation_accuracy(val_loader)
        self.log('val_accuracy', val_acc, prog_bar=True)
        print(f'Validation Accuracy: {val_acc:.4f}')

    def on_train_epoch_end(self, x): # outputs
        avg_loss = torch.stack([x['train_loss'] for x in outputs]).mean()
        self.log('train_loss', avg_loss)
        train_acc = self.accuracy(train_loader)
        self.log('train_accuracy', train_acc, prog_bar=True)
        print(f'Training Accuracy: {train_acc:.4f}')
    '''

# Define transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# New Load dataset
train_dataset = ImageFolder('/content/inaturalist_12K/train', transform=transform)
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_data, val_data = random_split(train_dataset, [train_size, val_size])
test_data = ImageFolder('/content/inaturalist_12K/val', transform=transform)


# Old Load dataset
'''
train_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/Colab Notebooks/nature_12K/inaturalist_12K/train', transform=transform)
test_data =  datasets.ImageFolder(root='/content/drive/MyDrive/Colab Notebooks/nature_12K/inaturalist_12K/val', transform=transform)
'''

# Create data loaders

def create_data_loaders(train_data, val_data, batch_size, shuffle_train=True, pin_memory=True, num_workers=2):
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=shuffle_train, pin_memory=pin_memory, num_workers=num_workers)
    val_loader = DataLoader(val_data, batch_size=batch_size, pin_memory=pin_memory, num_workers=num_workers)
    return train_loader, val_loader

train_loader, val_loader = create_data_loaders(train_data, val_data, batch_size = 32, shuffle_train=True)
'''
batch_size= 32
train_loader = DataLoader(train_data, batch_size = batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size= batch_size)



# Initialize model
model = ECNN(input_shape=(3, 224, 224), num_classes=10, num_filters=[32,32,32,32,32], filter_size=[3,3,3,3,3], dense_neurons=128,batch_normalization= True, batch_size = 32,dropout=0.2 ,activation='relu')
#x = torch.randn(1,3,224,224)
#x = torch.randn(32,3,224,224)
#a =model(x)
#print(a.shape)

# Initialize Lightning trainer with GPU support
trainer = pyligtining.Trainer(max_epochs=1, accelerator="auto") # not GPU but accelerator

# Train the model
trainer.fit(model, train_loader, val_loader)

print(model.validation_accuracy(train_loader))
print(model.validation_accuracy(val_loader))

# Test the model
#trainer.test(model, val_loader)
#print(model.test_accuracy(val_loader))
'''

'\nbatch_size= 32\ntrain_loader = DataLoader(train_data, batch_size = batch_size, shuffle=True)\nval_loader = DataLoader(val_data, batch_size= batch_size)\n\n\n\n# Initialize model\nmodel = ECNN(input_shape=(3, 224, 224), num_classes=10, num_filters=[32,32,32,32,32], filter_size=[3,3,3,3,3], dense_neurons=128,batch_normalization= True, batch_size = 32,dropout=0.2 ,activation=\'relu\')\n#x = torch.randn(1,3,224,224)\n#x = torch.randn(32,3,224,224)\n#a =model(x)\n#print(a.shape)\n\n# Initialize Lightning trainer with GPU support\ntrainer = pyligtining.Trainer(max_epochs=1, accelerator="auto") # not GPU but accelerator\n\n# Train the model\ntrainer.fit(model, train_loader, val_loader)\n\nprint(model.validation_accuracy(train_loader))\nprint(model.validation_accuracy(val_loader))\n\n# Test the model\n#trainer.test(model, val_loader)\n#print(model.test_accuracy(val_loader))\n'

# Sweep Iteration 1

In [6]:
# wandb sweep
!pip install --upgrade wandb
!wandb login 3c21150eb43b007ee446a1ff6e87f640ec7528c4 #my API key for wandb login


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [10]:
from torchsummary import summary
from pytorch_lightning.callbacks import EarlyStopping
import torch.optim as optim
import pytorch_lightning as pyligtining
import wandb

sweep_config = {
    'method': 'bayes', #grid, random,bayes
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'
    },
    'parameters': {
        'activation': {
            'values': ['relu','gelu','elu','silu']
        },
        'num_filter': {
            'values': [[32,32,32,32,32] ,[64,64,64,64,64] ,[128,128,128,128,128],
                       [64,128,256,512,1024]]
        },
        'filter_size':{
            'values':[[3,3,3,5,5],[5,5,5,5,5],[3,3,3,3,3]]
        },
        'dropout':{
            'values':[0.2,0.3]
        },
        'batch_normal':{
            'values':[True,False]
        },
        'batch_size':{
            'values':[32,64]
        },
        'data_augmentation':{
            'values':[True,False]
        },
        'input_shape':{
            'values':[(3, 224, 224)]
        },
        'num_classes':{
            'values': [10]
        },
    }
}

sweep_id = wandb.sweep(sweep_config, entity='ed23d015', project="DL_Assignment_2")

def sweep_train():
  # Default hyper-parameters
  config_defaults = {
      'activation':'relu',
      'num_filter':[32,32,32,32,32],
      'filter_size':[3,3,3,3,3],
      'dropout':0.3,
      'batch_normal':True,
      'batch_sizze': 32,
      'data_augmentation':True,
      'input_shape':(3, 224, 224),
      'num_classes': 10,
  }

  # Initialize wandb run
  wandb.init(project='DL_Assignment_2', entity='ed23d015',config=config_defaults)
  wandb.run.name = 'act:'+ str(wandb.config.activation)+' ;filter:'+str(wandb.config.num_filter)+ ' ;ker:'+str(wandb.config.filter_size)+ ' ;drop:'+str(wandb.config.dropout)+' ;b_n:'+str(wandb.config.batch_normal) +' ;b_s:'+str(wandb.config.batch_size)+' ;d_a:'+str(wandb.config.data_augmentation)

  config = wandb.config
  activation = config.activation
  num_filter = config.num_filter
  filter_size = config.filter_size
  dropout = config.dropout
  batch_normal = config.batch_normal
  batch_size = config.batch_size
  data_augmentation = config.data_augmentation
  input_shape = config.input_shape
  num_classes = config.num_classes

  train_loader, val_loader = create_data_loaders(train_data, val_data, batch_size, shuffle_train=True, pin_memory=True, num_workers = 2)

  model = ECNN(input_shape=input_shape, num_classes=num_classes, num_filters=num_filter, filter_size=filter_size, dense_neurons=128,batch_normalization= batch_normal, batch_size = batch_size, dropout=dropout ,activation=activation)

  # Initialize Lightning trainer with GPU support
  trainer1 = pl.Trainer(max_epochs=10, accelerator = "gpu" if torch.cuda.is_available() else "cpu", devices = 1 if torch.cuda.is_available() else 2, log_every_n_steps=30, callbacks=[EarlyStopping(monitor='val_loss')])

  # Train the model
  trainer1.fit(model, train_loader, val_loader)

  # Calculating losses and accuracies
  train_loss = model.training_step(train_loader)
  val_loss = model.validation_step(val_loader)
  train_accuracy = model.validation_accuracy(train_loader)
  val_accuracy = model.validation_accuracy(val_loader)


  # Log metrics to wandb
  wandb.log({"epoch": epoch + 1, "train_loss": train_loss, "train_accuracy": train_accuracy, "model.": val_loss, "val_accuracy": val_accuracy})

  # Print training progress
  print(f"Epoch {epoch + 1}/{trainer1.max_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%")


  '''
  # It's training time

  model = ECNN(input_shape=(3, 224, 224), num_classes=10, num_filters=num_filter, filter_size=filter_size, dense_neurons=128, batch_normalization=batch_normal, batch_size = batch_size  dropout=dropout, activation=activation)

  # Initialize Lightning trainer with GPU support
  #trainer = pyligtining.Trainer(max_epochs=10, accelerator="auto") # not GPU but accelerator
  trainer = pl.Trainer(max_epochs=10, gpus=1 if torch.cuda.is_available() else None, progress_bar_refresh_rate=30, callbacks=[EarlyStopping(monitor='val_loss')])


  # Train the model
  trainer.fit(model, train_loader, val_loader)

  # Test the model
  #trainer.test()

  # Test the model
  trainer.test(test_dataloaders=val_loader)
  '''
# From here

  '''
  train_loader, val_loader = create_data_loaders(train_data, val_data, batch_size, shuffle_train=True, pin_memory=True, num_workers = 2)
  # It's training time
  model = ECNN(input_shape=input_shape, num_classes=num_classes, num_filters=num_filter, filter_size=filter_size, dense_neurons=128, batch_normalization=batch_normal, batch_size=batch_size , dropout=dropout, activation=activation)


  print('batch_size: ' , batch_size)
  # Define loss function and optimizer
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=0.001)


  # Initialize Lightning trainer with GPU support
  #trainer1 = pl.Trainer(max_epochs=10, accelerator="gpu" if torch.cuda.is_available() else "cpu", progress_bar_refresh_rate=30, callbacks=[EarlyStopping(monitor='val_loss')])
  trainer1 = pl.Trainer(max_epochs=10, accelerator = "gpu" if torch.cuda.is_available() else "cpu", devices = 1 if torch.cuda.is_available() else 2, log_every_n_steps=30, callbacks=[EarlyStopping(monitor='val_loss')])

  # Initialize Lightning trainer
  #trainer1 = pyligtining.Trainer(max_epochs=10, accelerator='auto')#'cpu'
  #trainer1 = pl.Trainer(max_epochs=10, gpus = 1 if torch.cuda.is_available() else 0, test_dataloaders=val_loader)
  #print('Check point 1')

  # Training loop
  for epoch in range(trainer1.max_epochs):
      # Train the model
      model.train()
      train_loss = 0.0
      correct_train = 0
      total_train = 0
      #print('Check point 2')
      for images, labels in train_loader:
          optimizer.zero_grad()
          outputs = model(images)
          #print('Check point 2.1')
          loss = criterion(outputs, labels)
          #print('Check point 2.2')
          loss.backward()
          #print('Check point 2.3')
          optimizer.step()
          #print('Check point 3')
          train_loss += loss.item()
          _, predicted = torch.max(outputs, 1)
          total_train += labels.size(0)
          correct_train += (predicted == labels).sum().item()

      train_accuracy = 100 * correct_train / total_train
      train_loss /= len(train_loader)
      #print('Check point 4')
      # Validate the model
      model.eval()
      val_loss = 0.0
      correct_val = 0
      total_val = 0
      #print('Check point 5')
      with torch.no_grad():
          for images, labels in val_loader:
              outputs = model(images)
              loss = criterion(outputs, labels)
              val_loss += loss.item()
              _, predicted = torch.max(outputs, 1)
              total_val += labels.size(0)
              correct_val += (predicted == labels).sum().item()
              #print('Check point 6')
      val_accuracy = 100 * correct_val / total_val
      val_loss /= len(val_loader)

      # Log metrics to wandb
      wandb.log({"epoch": epoch + 1, "train_loss": train_loss, "train_accuracy": train_accuracy, "val_loss": val_loss, "val_accuracy": val_accuracy})

      # Print training progress
      print(f"Epoch {epoch + 1}/{trainer1.max_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%")

  # Test the model
  trainer1.test(test_dataloaders=val_loader)
  #trainer1.test()
  '''
#RUNNING THE SWEEP
wandb.agent(sweep_id, function=sweep_train, count=120) #change the number of count here

#To here

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: b3qcwh7b
Sweep URL: https://wandb.ai/ed23d015/DL_Assignment_2/sweeps/b3qcwh7b


wandb: Agent Starting Run: otmejofu with config:
wandb: 	activation: gelu
wandb: 	batch_normal: True
wandb: 	batch_size: 64
wandb: 	data_augmentation: True
wandb: 	dropout: 0.2
wandb: 	filter_size: [5, 5, 5, 5, 5]
wandb: 	input_shape: [3, 224, 224]
wandb: 	num_classes: 10
wandb: 	num_filter: [64, 128, 256, 512, 1024]
Exception in thread ChkStopThr:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_run.py", line 286, in check_stop_status
Exception in thread IntMsgThr:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/wandb/

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:lightning_fabric.utilities.distributed:Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/2
INFO:lightning_fabric.utilities.distributed:Initializing distributed: GLOBAL_RANK: 1, MEMBER: 2/2


wandb: ERROR Run otmejofu errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/usr/local/lib/python3.10/dist-packages/wandb/agents/pyagent.py", line 308, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "<ipython-input-10-345e65ee6910>", line 84, in sweep_train
wandb: ERROR     trainer1.fit(model, train_loader, val_loader)
wandb: ERROR   File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/trainer.py", line 544, in fit
wandb: ERROR     call._call_and_handle_interrupt(
wandb: ERROR   File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/call.py", line 43, in _call_and_handle_interrupt
wandb: ERROR     return trainer.strategy.launcher.launch(trainer_fn, *args, trainer=trainer, **kwargs)
wandb: ERROR   File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/strategies/launchers/multiprocessing.py", line 144, in launch
wandb: ERROR     while not process_context.join():
wandb: ERROR   File "/usr/local/li

...

wandb: ERROR Run eov545r9 errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/usr/local/lib/python3.10/dist-packages/wandb/agents/pyagent.py", line 308, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "<ipython-input-10-345e65ee6910>", line 84, in sweep_train
wandb: ERROR     trainer1.fit(model, train_loader, val_loader)
wandb: ERROR   File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/trainer.py", line 544, in fit
wandb: ERROR     call._call_and_handle_interrupt(
wandb: ERROR   File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/call.py", line 43, in _call_and_handle_interrupt
wandb: ERROR     return trainer.strategy.launcher.launch(trainer_fn, *args, trainer=trainer, **kwargs)
wandb: ERROR   File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/strategies/launchers/multiprocessing.py", line 144, in launch
wandb: ERROR     while not process_context.join():
wandb: ERROR   File "/usr/local/li

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:lightning_fabric.utilities.distributed:Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/2
INFO:lightning_fabric.utilities.distributed:Initializing distributed: GLOBAL_RANK: 1, MEMBER: 2/2


wandb: ERROR Run wvbq4id4 errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/usr/local/lib/python3.10/dist-packages/wandb/agents/pyagent.py", line 308, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "<ipython-input-10-345e65ee6910>", line 84, in sweep_train
wandb: ERROR     trainer1.fit(model, train_loader, val_loader)
wandb: ERROR   File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/trainer.py", line 544, in fit
wandb: ERROR     call._call_and_handle_interrupt(
wandb: ERROR   File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/call.py", line 43, in _call_and_handle_interrupt
wandb: ERROR     return trainer.strategy.launcher.launch(trainer_fn, *args, trainer=trainer, **kwargs)
wandb: ERROR   File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/strategies/launchers/multiprocessing.py", line 144, in launch
wandb: ERROR     while not process_context.join():
wandb: ERROR   File "/usr/local/li

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7d6e15b67850>> (for post_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

In [ ]:
import torch
import pytorch_lightning as pl
import wandb
from torch.utils.data import DataLoader

class MyLightningModule(pl.LightningModule):
    def __init__(self, model, criterion, optimizer):
        super().__init__()
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.criterion(y_hat, y)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.criterion(y_hat, y)
        self.log('val_loss', loss)
        return loss

    def configure_optimizers(self):
        return self.optimizer

def sweep_train():
    # Initialize wandb run
    wandb.init(project='DL_Assignment_2', entity='ed23d015')

    # Fetch the default configuration from wandb
    config = wandb.config

    # Set up data loaders based on configuration
    train_loader = DataLoader(train_data, batch_size=config.batch_size, shuffle=True, pin_memory=True, num_workers=4)
    val_loader = DataLoader(val_data, batch_size=config.batch_size, pin_memory=True, num_workers=4)

    # Initialize model, criterion, and optimizer
    model = ...
    criterion = ...
    optimizer = ...

    # Initialize PyTorch Lightning module
    lightning_module = MyLightningModule(model, criterion, optimizer)

    # Initialize PyTorch Lightning Trainer
    trainer = pl.Trainer(max_epochs=10, gpus=1 if torch.cuda.is_available() else None, progress_bar_refresh_rate=30, callbacks=[pl.callbacks.EarlyStopping(monitor='val_loss')])

    # Train the model
    trainer.fit(lightning_module, train_loader, val_loader)

    # Test the model
    trainer.test(lightning_module, val_loader)

# Run the wandb sweep
sweep_id = wandb.sweep(sweep_config, entity='ed23d015', project="DL_Assignment_2")
wandb.agent(sweep_id, function=sweep_train, count=120)


In [ ]:
def sweep_train():
    # Initialize wandb run
    wandb.init(project='DL_Assignment_2', entity='ed23d015')

    # Fetch the default configuration from wandb
    config = wandb.config

    # Set up data loaders based on configuration
    train_loader = DataLoader(train_data, batch_size=config.batch_size, shuffle=True, pin_memory=True, num_workers=4)
    val_loader = DataLoader(val_data, batch_size=config.batch_size, pin_memory=True, num_workers=4)

    # Initialize model, criterion, and optimizer
    model = ...
    criterion = ...
    optimizer = ...

    # Initialize PyTorch Lightning module
    lightning_module = MyLightningModule(model, criterion, optimizer)

    # Initialize PyTorch Lightning Trainer
    trainer = pl.Trainer(max_epochs=10, gpus=1 if torch.cuda.is_available() else None, progress_bar_refresh_rate=30, callbacks=[pl.callbacks.EarlyStopping(monitor='val_loss')])

    # Train the model
    trainer.fit(lightning_module, train_loader, val_loader)

    # Test the model
    trainer.test(lightning_module, val_loader)

# Run the wandb sweep
sweep_id = wandb.sweep(sweep_config, entity='ed23d015', project="DL_Assignment_2")
wandb.agent(sweep_id, function=sweep_train, count=120)

#2nd try

In [ ]:
import torch

# Ensure model and data loaders are moved to appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
for epoch in range(trainer1.max_epochs):
    model.train()
    train_loss = 0.0
    correct_train = 0
    total_train = 0

    # Use tqdm for progress bar
    for images, labels in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{trainer1.max_epochs}'):
        # Move data to device
        images, labels = images.to(device), labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)

        # Calculate loss
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()

        # Update training loss
        train_loss += loss.item()

        # Calculate training accuracy
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    # Calculate training accuracy
    train_accuracy = 100 * correct_train / total_train
    train_loss /= len(train_loader)

    # Validation
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    # No need for gradients during validation
    with torch.no_grad():
        for images, labels in val_loader:
            # Move data to device
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)

            # Calculate loss
            loss = criterion(outputs, labels)

            # Update validation loss
            val_loss += loss.item()

            # Calculate validation accuracy
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    # Calculate validation accuracy
    val_accuracy = 100 * correct_val / total_val
    val_loss /= len(val_loader)

    # Log metrics to wandb
    wandb.log({"epoch": epoch + 1, "train_loss": train_loss, "train_accuracy": train_accuracy, "val_loss": val_loss, "val_accuracy": val_accuracy})

    # Print training progress
    print(f"Epoch {epoch + 1}/{trainer1.max_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%")

# Test the model
trainer1.test(test_dataloaders=val_loader)
